# Financial Functions - Stock Market Analysis

## Open AI Token

In [ ]:
token = "ENTER-YOUR-OPENAI-API-KEY"

## This gives us the stock name of a company asked in general english terms

In [ ]:
!pip install crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.2 MB/s eta 0:00

In [ ]:
from crewai.tools import tool

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=token)

@tool("get_stock_name_from_english_terms")
def get_stock_name_from_english_terms(stock_symbol:str):
    """Fetches the current price of the stock"""
    prompt = f"""Respond with a single word: the exact stock ticker symbol for this company: {stock_symbol}, suitable for use with the Yahoo Finance API — no explanations or extra text."""

    completion = client.chat.completions.create(
        model="gpt-4o",
        store=True,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content

In [ ]:
!pip install --upgrade yfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.8/113.8 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.56
    Uninstalling yfinance-0.2.56:
      Successfully uninstalled yfinance-0.2.56


## Current Stock Price of a company

In [ ]:
import yfinance as yf
from datetime import datetime

@tool("get_current_price")
def get_current_price(stock_name: str):
    """Fetches the current price of the stock"""
    stock = yf.Ticker(stock_name)
    try:
        current_price = stock.fast_info['lastPrice']
        return current_price
    except KeyError:
        return "Price info not available"

# def get_current_price(stock_name:str):
#     stock = yf.Ticker(stock_name)
#     current_date = datetime.now().strftime("%Y-%m-%d")
#     # Get historical data (daily)
#     history = stock.history(start="2024-04-18", end="2024-04-18")
#     current_price_stock = history.to_json(orient='records')
#     return current_price_stock

## Financial Reports of the Company

In [ ]:
@tool("get_financials")
def get_financials(stock_name:str):
  """gets the financials reports of company"""
  ticker = yf.Ticker(stock_name)
  result = ticker.financials
  result = result.dropna(how='all').dropna(axis=1, how='all')
  latest_col = result.columns[0]
  data_dict = result[latest_col].dropna().to_dict()
  return data_dict

## Currency of the Stock and Company profile

In [ ]:
@tool("get_currency_stock_and_profile")
def get_currency_stock_and_profile(stock_name:str):
  """Fetches the currency and profile of the stock"""
  ticker = yf.Ticker(stock_name)
  company_info = ticker.info.get('longBusinessSummary')
  currency_stock = ticker.info['currency']
  return {'company_info': company_info, 'currency_stock': currency_stock}

## Get Latest News of the Company

In [ ]:
import yfinance as yf
from bs4 import BeautifulSoup

@tool("get_recent_stock_news")
def get_recent_stock_news(stock_name:str, limit:int):
    """gets the latest news and updates of a company"""
    ticker = yf.Ticker(stock_name)
    news_items = ticker.news
    result = []
    limit = 5

    for item in news_items[:limit]:
        content = item.get('content', {})
        title = content.get('title', '')
        pub_date = content.get('pubDate', '')
        raw_description = content.get('description', '')

        # Remove HTML tags
        soup = BeautifulSoup(raw_description, "html.parser")
        clean_description = soup.get_text(separator=" ", strip=True)

        result.append({
            'headline': title,
            'date': pub_date
        })

    return result

In [ ]:

import requests

@tool("get_analyst_opinion_requests")
def get_analyst_opinion_requests(stock_name: str):
    """Fetches the stock market analyst's opinion for a given company"""
    url = "https://serpapi.com/search.json"
    params = {
        "engine": "google",
        "q": f"{stock_name} stock analyst rating OR buy recommendation site:yahoofinance.com OR site:marketwatch.com OR site:economictimes.indiatimes.com",
        "tbm": "nws",
        "api_key": "ENTER-YOUR-SERPAPI-API-KEY"
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data.get("news_results", [])

In [ ]:
@tool("format_analyst_snippets")
def format_analyst_snippets(news_results:list, stock_name:str):
    """Formats the stock market analyst's opinion for a given company's output"""
    if not news_results:
        return f"No analyst commentary found for {stock_name}."

    summary_lines = []
    for item in news_results:
        title = item.get("title", "No title")
        snippet = item.get("snippet", "")
        source = item.get("source", "")
        summary_lines.append(f"- [{source}] {title}: {snippet}")

    return "\n".join(summary_lines)


# NLP Techniques

## Hugging Face token

In [ ]:
hf_token = 'ENTER-YOUR-HF-API-KEY'

## Sentiment Analysis - FinBERT

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

sentiment_tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert", use_auth_token=hf_token)
sentiment_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", use_auth_token=hf_token)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
from transformers import pipeline
sentiment_nlp = pipeline("sentiment-analysis", model=sentiment_model, tokenizer=sentiment_tokenizer)

Device set to use cpu


In [ ]:
@tool("analyze_sentiment")
def analyze_sentiment(text:str):
    """Analyzes the sentiment of a given text."""
    result = {}
    analysis = sentiment_nlp(text)
    if len(analysis) > 0:
        result['headline'] = text
        result['sentiment'] = analysis[0]['label']
        return result
    else:
        return "No sentiment detected" # Returns a sentiment score between -1 and 1

## Named Entity Recognition

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER", use_auth_token=hf_token)
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER", use_auth_token=hf_token)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
ner_model = pipeline("ner", model=model, tokenizer=tokenizer)

@tool("extract_named_entities")
def extract_named_entities(text:str):
    """Extracts named entities in a given text."""
    res = {}
    result = ner_model(text)
    res['text'] = text
    res['Named Entities'] = result
    return res

Device set to use cpu


# LLM Agent Initialisation

In [ ]:
import os

In [ ]:
STOCK="Analyze the stock GameStop"

In [ ]:
os.environ["OPENAI_API_KEY"] = token
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

In [ ]:
from crewai import Agent, Task
from typing import Any, Dict, List, Optional

In [ ]:
get_stock_name_agent = Agent(
    role='Get the stock name of the company',
    goal= f"Respond with a single word: the exact stock ticker symbol for the company asked in simple english: {STOCK}, suitable for use with the Yahoo Finance API — no explanations or extra text.",
    backstory=f"The outcome of this task is a single word: the exact stock ticker symbol for the company asked in simple english terms with the help of the tool, suitable for use with the Yahoo Finance API — no explanations or extra text.",
    tools=[get_stock_name_from_english_terms],
    memory=True, # Enable memory
    verbose=True,
    max_rpm=None, # No limit on requests per minute
    max_iter=10, # Default value for maximum iterations
    llm ="gpt-4o"
)

In [ ]:
get_company_profile_and_currecy_agent = Agent(
    role='Company Profile and Currency aggregator',
    goal=f"Fetches the currency and profile of the stock",
    backstory=f"The outcome of this task is a structured response summarizing the financial data of the specified company. This includes: Currency of the stock: The currency of the stock. Company Profile: A brief summary of the company's history, values, and mission.",
    tools=[get_currency_stock_and_profile],
    memory=True, # Enable memory
    verbose=True,
    max_rpm=None, # No limit on requests per minute
    max_iter=10, # Default value for maximum iterations
    llm ="gpt-4o"
)

In [ ]:
get_current_price_agent = Agent(
    role='Current Price aggregator',
    goal=f"Fetches the current price of the stock from the stock name.",
    backstory=f"The outcome of this task is the current price of the stock",
    tools=[get_current_price],
    memory=True, # Enable memory
    verbose=True,
    max_rpm=None, # No limit on requests per minute
    max_iter=10, # Default value for maximum iterations
    llm ="gpt-4o"
)

In [ ]:
financial_agent = Agent(
  role='Financial Data Aggregator',
  goal = f"Retrieve financial reports (e.g., annual and quarterly) of the company",
  # goal=' Retrieve name of the stock, currency and compaof the stock and company profile, current stock price, key performance metrics, and financial reports (e.g., annual and quarterly).',
  backstory="A seasoned expert adept at extracting and organizing all relevant financial details for thorough analysis.",
  tools=[get_financials],
  memory=True, # Enable memory
  verbose=True,
  max_rpm=None, # No limit on requests per minute
  max_iter=10, # Default value for maximum iterations
  llm ="gpt-4o"
)

In [ ]:
news_agent = Agent(
  role='News Aggregator',
  goal='Gather, filter, and provide recent news, analyze sentiments and updates about the company, focusing on events that impact stock performance or market perception.',
  backstory=' A proficient news-fetching agent skilled at sourcing relevant, actionable information from credible news outlets and analysing sentiments associated with the news to aid in detailed company analysis.',
  tools=[get_recent_stock_news, analyze_sentiment],
  memory=True, # Enable memory
  verbose=True,
  max_rpm=None, # No limit on requests per minute
  max_iter=10, # Default value for maximum iterations
  llm ="gpt-4o"
)

In [ ]:
stock_market_analyst_agent = Agent(
  role='Stock Market Analyst',
  goal='Deliver sharp, data-driven insights into the financial health, competitive positioning, and market outlook of the company and extract named entities from the final text.',
  backstory='A seasoned stock market analyst with a deep understanding of global financial markets and sector-specific trends. Regularly tracks breaking news, analyst upgrades/downgrades, and macroeconomic indicators to anticipate stock movements.',
  tools=[get_analyst_opinion_requests, format_analyst_snippets, extract_named_entities],
  memory=True, # Enable memory
  verbose=True,
  max_rpm=None, # No limit on requests per minute
  max_iter=10, # Default value for maximum iterations
   llm ="gpt-4o"
)

In [ ]:
analysis_agent = Agent(
  role='Stock Analyst',
  goal='Integrate data from financial, market, and news agents to perform comprehensive analysis and deliver actionable Buy/Sell/Hold recommendations based on insights along with consolidated analysis of the stock',
  backstory='A strategic thinker proficient in combining multifaceted information to derive meaningful recommendations, balancing risk and opportunity in stock investments.',
  memory=True, # Enable memory
  verbose=True,
  max_rpm=None, # No limit on requests per minute
  max_iter=10, # Default value for maximum iterations,
   llm ="gpt-4o"
)

In [ ]:
get_stock_name_agent_task = Task(
    description=f"You are an agent responsible for fetching the stock name of the company from basic english sentence from the user",
    expected_output=f"The outcome of this task is a single word: the exact stock ticker symbol for the company asked in simple english terms: {STOCK}, suitable for use with the Yahoo Finance API — no explanations or extra text. So the value from the agent is assigned to STOCK",
    agent=get_stock_name_agent
)

In [ ]:
get_company_profile_and_currecy_task = Task(
    description=f"You are an agent responsible for fetching the currency and company profile of the stock",
    expected_output=f"The outcome of this task is a structured response summarizing the financial data of the specified company. This includes: Currency of the stock: The currency of the stock. Company Profile: A brief summary of the company's history, values, and mission.",
    agent=get_company_profile_and_currecy_agent
)

In [ ]:
get_current_price_task = Task(
    description = "You are an agent responsible for fetching the current price of the stock",
    expected_output = f"The outcome of this task is the current price of the stock",
    agent=get_current_price_agent
)

In [ ]:
get_financial_task = Task(
    description=f"""
You are an agent responsible for fetching the stock name, current price of the stock, curreny and company profile and lastly financial reports of a given company. You have access to the following tools to accomplish this task:
get_financials: gets the financials reports of company for the last year for the stock.
Use all the tools at your disposal to fetch comprehensive financial data for the specified company and provide a structured response summarizing your findings.
""",
expected_output="""The outcome of this task is a structured response summarizing the financial data of the specified company. This includes:
Financial Reports: A summary of the financial reports for the last year, including key metrics such as revenue, profit, and other relevant financial indicators. """,
agent=financial_agent,
)

In [ ]:
get_latest_news_task = Task(
    description=f"As a news reporting task your job is to provide up-to-date information about the specified company. You have access to web scraping tool that can fetch news with its date. For the stock - {STOCK} and also the sentiment analysis tool that provides sentiment based on comapny's headlines and descriptions",
    expected_output="The outcome of this task is a structured response summarizing the latest news and updates about the specified company along with the date and sentiments associated with it.",
    agent=news_agent,
)

In [ ]:
stock_market_analyst_task = Task(
    description=f"Analyze the latest news, analyst opinions, and market sentiment for a given stock symbol using this stock market analyst tool. Use the available tools to extract relevant articles, summaries, and expert commentary from trusted sources and format it. Synthesize this information into a concise, informative report",
    expected_output="""A well-structured text report containing:\n
        - A summary of recent analyst opinions and their tone\n"
        - Highlights from news headlines with relevant snippets and sources\n"
        - An assessment of the company's market position and any notable risks or opportunities\n"
        - A concluding sentiment based on the information gathered""",
    agent=stock_market_analyst_agent)

In [ ]:
analysis_task = Task(
    description="As an task responsible for performing a consolidated analysis of a given stock, your job is to provide a comprehensive evaluation based on the financial results, market research analysis, and the latest news from other agents. You will consider all relevant points and provide a recommendation on whether to buy, sell, or perform more analysis on the stock. Your analysis should be thorough and well-reasoned",

    expected_output="""You are a financial analysis agent tasked with providing a comprehensive evaluation of a given stock.
Your analysis must be based on inputs from financial results, market research insights, stock market analyst opinions, and the latest news sentiment.
Structure your response covering:
1. Begin with the average price target followed by the high and low forecasts.
2. Clearly state the number of analysts, time frame (e.g., “last 3 months”), and company name.
3. Include the upside percentage compared to the last trading price.
4. Present in a neutral and professional tone, supporting either a Buy, Hold, or Sell recommendation.
Use a concise and professional tone, and organize your response in the above manner.

Example format of output of your response:
Based on 42 Wall Street analysts offering 12-month price targets for Nvidia in the last 3 months, the average price target is $168.43 with a high forecast of $220.00 and a low forecast of $120.00. The average price target represents a 51.73% change from the last price of $111.01.
""",

    agent=analysis_agent,
)


In [ ]:
# analysis_task = Task(
#     description="As an task responsible for performing a consolidated analysis of a given stock, your job is to provide a comprehensive evaluation based on the financial results, market research analysis, and the latest news from other agents. You will consider all relevant points and provide a recommendation on whether to buy, sell, or perform more analysis on the stock. Your analysis should be thorough and well-reasoned",
#     expected_output=""" The outcome of this task is a one word response recommending whether to buy/sell/hold the stock. This includes:
# Recommendation: A clear one word recommendation on whether to buy, sell, or hold the stock with the name of the stock.""",
#     agent=analysis_agent,
# )

In [ ]:
from crewai import Crew,Process

In [ ]:
stock_crew = Crew(
  agents=[get_stock_name_agent, get_current_price_agent,get_company_profile_and_currecy_agent, financial_agent,news_agent, stock_market_analyst_agent,analysis_agent],
  tasks=[get_stock_name_agent_task, get_company_profile_and_currecy_task, get_current_price_task, get_financial_task,get_latest_news_task,stock_market_analyst_task,analysis_task],
  process=Process.sequential
)

In [ ]:
result = stock_crew.kickoff()

# Agent: Get the stock name of the company
## Task: You are an agent responsible for fetching the stock name of the company from basic english sentence from the user


# Agent: Get the stock name of the company
## Thought: Thought: I need to determine the stock ticker symbol for GameStop to fulfill the user's request.
## Using tool: get_stock_name_from_english_terms
## Tool Input: 
"{\"stock_symbol\": \"GameStop\"}"
## Tool Output: 
GME


# Agent: Get the stock name of the company
## Final Answer: 
GME


# Agent: Company Profile and Currency aggregator
## Task: You are an agent responsible for fetching the currency and company profile of the stock


# Agent: Company Profile and Currency aggregator
## Thought: Thought: I need to use the available tool to fetch the currency and company profile for the stock ticker "GME."
## Using tool: get_currency_stock_and_profile
## Tool Input: 
"{\"stock_name\": \"GME\"}"
## Tool Output: 
{'company_info': 'GameStop Corp., a specialty retailer, provid

In [ ]:
wallstreet_res = "Based on 33 Wall Street analysts offering 12 month price targets for Apple in the last 3 months. The average price target is $234.13 with a high forecast of $300.00 and a low forecast of $167.88. The average price target represents a 10.18% change from the last price of $212.50."

In [ ]:
generated_res = """Based on 42 Wall Street analysts offering 12-month price targets for Apple Inc. (AAPL) in the last 3 months, the average price target is $237.62 with a high forecast of $250.00 and a low forecast of $225.00. The average price target represents an 11.84% change from the last price of $212.50 USD.

The financial results indicate a robust performance for Apple, with a substantial net income of $93.736 billion and strong investment in R&D amounting to $31.370 billion, supporting continuous innovation across its product lines. The latest news articles present both potential challenges and opportunities for Apple. A notable ruling could impact revenue from non-App Store sales commissions, yet its impact on the overall revenue is still ambiguous. On the other hand, Apple's recent product launches and strategic positioning strengthen its competitive edge.

Analysts maintain an "Overweight" rating, suggesting expectations for the stock to outperform the market. Risks include potential legal constraints and macroeconomic uncertainties affecting consumer spending, but opportunities are evident in Apple's expansion of services and market innovation, underpinning its growth trajectory."""

## Evaluation


In [ ]:
pip install rouge-score bert-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.6 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24

In [ ]:
from rouge_score import rouge_scorer
from bert_score import score as bert_score

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(generated_res, wallstreet_res)

# Display ROUGE results
for key, value in rouge_scores.items():
    print(f"{key}: Precision: {value.precision:.4f}, Recall: {value.recall:.4f}, F1-score: {value.fmeasure:.4f}")


rouge1: Precision: 0.8545, Recall: 0.2527, F1-score: 0.3900
rouge2: Precision: 0.7222, Recall: 0.2108, F1-score: 0.3264
rougeL: Precision: 0.8364, Recall: 0.2473, F1-score: 0.3817


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `llamatoken_srg` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `lla

In [ ]:
!huggingface-cli download bert-base-uncased --local-dir ./bert-model

Fetching 16 files:   0% 0/16 [00:00<?, ?it/s]Downloading '.gitattributes' to 'bert-model/.cache/huggingface/download/wPaCkH-WbT7GsmxMKKrNZTV4nSM=.505a7adf8be9e5fdf06aabbfbe9046e6c811f91b.incomplete'

LICENSE: 100% 11.4k/11.4k [00:00<00:00, 36.0MB/s]
Download complete. Moving file to bert-model/LICENSE

README.md: 100% 10.5k/10.5k [00:00<00:00, 36.2MB/s]

.gitattributes: 100% 491/491 [00:00<00:00, 3.20MB/s]
Download complete. Moving file to bert-model/.gitattributes
Download complete. Moving file to bert-model/README.md

config.json: 100% 570/570 [00:00<00:00, 3.83MB/s]
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Download complete. Moving file to bert-model/config.json
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better perf

In [ ]:
from bert_score import BERTScorer

# Define the local model path
local_model_path = "/content/bert-model"

# Initialize BERTScorer with the local model
scorer = BERTScorer(model_type=local_model_path, idf=False, num_layers=9, batch_size=1, lang="en")

# Compute BERTScore
P, R, F1 = scorer.score([generated_res], [wallstreet_res])

# Print results
print(f"BERTScore Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

BERTScore Precision: 0.5927, Recall: 0.8903, F1: 0.7116


In [ ]:
generated_res = """Based on 69 Wall Street analysts offering 12-month price targets for NVIDIA Corporation (NVDA) in the last 3 months, the average price target is $160.47 with a high forecast of $233.00 and a low forecast of $112.00. The average price target represents a 47.34% change from the last price of $108.92.

NVIDIA Corporation, a leader in computing infrastructure, particularly in graphics and computing technologies, has shown robust financial performance with a total revenue of $130.497 billion and an impressive net income of $72.880 billion. The company's strength in diverse sectors such as gaming, AI, and automotive, along with its noteworthy R&D investment of $12.914 billion, underscores its commitment to innovation.

Despite macroeconomic uncertainties and competitive pressures highlighted in certain negative news headlines, recent analyst recommendations predominantly maintain a positive stance. Evercore ISI's Mark Lipacis notably recommends buying NVDA stock ahead of earnings, reflecting strong confidence in NVIDIA's market position and future performance, even amidst potential market turmoil."""

In [ ]:
generated_res = "Based on 69 Wall Street analysts offering 12-month price targets for NVIDIA Corporation (NVDA) in the last 3 months, the average price target is $160.47 with a high forecast of $233.00 and a low forecast of $112.00. The average price target represents a 47.34% change from the last price of $108.92."

In [ ]:
wallstreet_res = "Based on 40 Wall Street analysts offering 12 month price targets for Nvidia in the last 3 months. The average price target is $167.09 with a high forecast of $200.00 and a low forecast of $120.00. The average price target represents a 53.41% change from the last price of $108.92."

In [ ]:
from bert_score import BERTScorer

# Define the local model path
local_model_path = "/content/bert-model"

# Initialize BERTScorer with the local model
scorer = BERTScorer(model_type=local_model_path, idf=False, num_layers=9, batch_size=1, lang="en")

# Compute BERTScore
P, R, F1 = scorer.score([generated_res], [wallstreet_res])

# Print results
print(f"BERTScore Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

BERTScore Precision: 0.9126, Recall: 0.9485, F1: 0.9302


In [ ]:
generated_res = """Based on 57 analysts offering a 12-month price target for Tesla, Inc. (TSLA) in recent evaluations, the average price target is $304.38 with a high forecast of approximately $350.00 and a low forecast of around $250.00. This average price target suggests a potential upside of approximately 7.85% from the last recorded trading price of $282.16 USD.

The evaluation of Tesla includes significant recent developments. A noteworthy event is the initiation by Tesla's board of the process to replace Elon Musk as CEO, as reported by The Wall Street Journal. This development carries a negative sentiment due to the uncertainty typically associated with changes in executive leadership.

Financially, Tesla exhibits robust figures with a total revenue of $97.69 billion and a net income of $7.13 billion. The company commands a strong presence in the automotive and energy generation markets, diversifying its revenue streams and strengthening its industry position against competitors, despite potential risks like leadership transitions, intensified market competition, and regulatory challenges."""

In [ ]:
generated_res = "Based on 57 analysts offering a 12-month price target for Tesla, Inc. (TSLA) in recent evaluations, the average price target is $304.38 with a high forecast of approximately $350.00 and a low forecast of around $250.00. This average price target suggests a potential upside of approximately 7.85% from the last recorded trading price of $282.16 USD."

In [ ]:
wallstreet_res = "Based on 37 Wall Street analysts offering 12 month price targets for Tesla in the last 3 months. The average price target is $283.69 with a high forecast of $450.00 and a low forecast of $115.00. The average price target represents a 0.54% change from the last price of $282.16."

In [ ]:
from bert_score import BERTScorer

# Define the local model path
local_model_path = "/content/bert-model"

# Initialize BERTScorer with the local model
scorer = BERTScorer(model_type=local_model_path, idf=False, num_layers=9, batch_size=1, lang="en")

# Compute BERTScore
P, R, F1 = scorer.score([generated_res], [wallstreet_res])

# Print results
print(f"BERTScore Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

BERTScore Precision: 0.7982, Recall: 0.8612, F1: 0.8285


In [ ]:
generated_res = """Based on recent analysis, 10 Wall Street analysts have offered 12-month price targets for GameStop Corp. (GME) within the last quarter. The average price target is $13.50, with a high forecast of $22.00 and a low forecast of $5.00. This average price target represents a downside of approximately 51.56% compared to the current trading price of $27.86 USD.

The financial performance of GameStop Corp. reveals some concerning aspects. Although the company reports a net income of $131.3 million, the operating income is negative at -$16.5 million, indicating inefficiencies in core operations. The gross profit of $1.113 billion against significant Selling, General, and Administrative Expenses of $1.13 billion further pinpoints the financial strain.

Recent news categorizes GameStop as an aggressive growth stock, still under the unproven leadership strategies of CEO Ryan Cohen, with ongoing discussions surrounding digital asset expansion. There is also significant market interest related to the company's position in emerging digital commerce and the gaming sector."""

In [ ]:
generated_res = "Based on recent analysis, 10 Wall Street analysts have offered 12-month price targets for GameStop Corp. (GME) within the last quarter. The average price target is $13.50, with a high forecast of $22.00 and a low forecast of $5.00. This average price target represents a downside of approximately 51.56% compared to the current trading price of $27.86 USD."

In [ ]:
wallstreet_res = "Based on 1 Wall Street analysts offering 12 month price targets for GameStop in the last 3 months. The average price target is $13.50 with a high forecast of $13.50 and a low forecast of $13.50. The average price target represents a -51.54% change from the last price of $27.86."

In [ ]:
from bert_score import BERTScorer

# Define the local model path
local_model_path = "/content/bert-model"

# Initialize BERTScorer with the local model
scorer = BERTScorer(model_type=local_model_path, idf=False, num_layers=9, batch_size=1, lang="en")

# Compute BERTScore
P, R, F1 = scorer.score([generated_res], [wallstreet_res])

# Print results
print(f"BERTScore Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

BERTScore Precision: 0.7964, Recall: 0.8688, F1: 0.8310
